## Memory

The Memory suite consists of **MemoryStore**, **MemorySource**, and **MemorySink**. Under the hood, the Memory suite points to an in-memory dictionary. Similarly, the **MemoryStore** is a just a wrapper around a paired **MemorySource** and **MemorySink**; as there is quite limited uses for just a **MemorySource** or a **MemorySink**, it is recommended to always use **MemoryStore**. The **MemoryStore** is intended for retrieving/searching and pushing STIX content to memory. It is important to note that all STIX content in memory is not backed up on the file system (disk), as that functionality is ecompassed within the **FileSystemStore**. However, the Memory suite does provide some utility methods for saving and loading STIX content to disk. **MemoryStore.save_to_file()** allows for saving all the STIX content that is in memory to a json file. **MemoryStore.load_from_file()** allows for loading STIX content from a json-formatted file. 


### Memory API

### MemorySource (snapshot)
* **get()**          - search/retrieve from memory most current STIX SDO/SRO via its ID
* **all_versions()** - search/retrieve from memory all versions of STIX SDO/SRO via its id
* **query()**        - search/retrieve from memory STIX SDO/SRO(s) via search filters
* **load_from_file()** - load STIX content into memory from file
 
### MemorySink (snapshot)
* **add()** - add a set of STIX SDO/SRO(s) to memory
* **save_to_file()** - save STIX content that is in memory to file
 
### MemoryStore (snapshot)

(super set of the MemorySource and MemorySink)
* **get()**
* **all_versions()**
* **query()**
* **load_from_file()**
* **save_to_file()**
* **add()**

A note on **load_from_file()** and **save()**. These methods both add STIX content to an internal dictionary (maintained by MemoryStore). STIX content that is to be added can be in the following forms: python STIX objects, python dicts (of valid STIX objects or Bundles), json-encoded strings (of valid STIX objects or Bundles), or a (python)list of any of the previously listed types. **MemoryStore** actually stores STIX content either as python STIX objects or as python dictionaries, reducing and converting any of the aforementioned types to one of those; and whatever form the STIX object is stored as , is what it will be returned as when queried or retrieved. Python STIX objects, and json-encoded strings (of STIX content) are stored as python STIX objects. Python dicts (of STIX objects) are stored as python dictionaries. This is done, as can be efficiently supported, in order to return STIX content in the form it was added to the **MemoryStore**. Also, for **load_from_file()**, STIX content is assumed to  be in json form within the file, individually or in a Bundle. 

A note on **save_to_file()**. This method dumps all STIX content that is in MemoryStore to the specified file. The file format will be json, and the STIX content will be within a STIX Bundle. ntoe, the the output form will be a json STIX Bundle regardless of the form that the individual STIX objects are stored(i.e. supplied) to the MemoryStore. 

### Memory Examples

#### MemoryStore

In [1]:
from stix2 import MemoryStore, Indicator

# create default MemoryStore
mem = MemoryStore()

# insert newly created indicator into memory
ind = Indicator(description="Crusades C2 implant",
                labels=["malicious-activity"],
                pattern="[file:hashes.'SHA-256' = '54b7e05e39a59428743635242e4a867c932140a999f52a1e54fa7ee6a440c73b']")

mem.add(ind)

# for visual purposes
print(mem.get(ind.id))


{
    "type": "indicator",
    "id": "indicator--340e9e90-80f7-4587-a1cb-172af51356e3",
    "created": "2017-09-24T19:32:32.257Z",
    "modified": "2017-09-24T19:32:32.257Z",
    "labels": [
        "malicious-activity"
    ],
    "description": "Crusades C2 implant",
    "pattern": "[file:hashes.'SHA-256' = '54b7e05e39a59428743635242e4a867c932140a999f52a1e54fa7ee6a440c73b']",
    "valid_from": "2017-09-24T19:32:32.257186Z"
}


In [2]:
from stix2 import Malware

# add multiple STIX objects into memory
ind2 = Indicator(description="Crusades stage 2 implant",
                 labels=["malicious-activity"],
                 pattern="[file:hashes.'SHA-256' = '70fa62fb218dd9d936ee570dbe531dfa4e7c128ff37e6af7a6a6b2485487e50a']")
ind3 = Indicator(description="Crusades stage 2 implant variant",
                 labels=["malicious-activity"],
                 pattern="[file:hashes.'SHA-256' = '31a45e777e4d58b97f4c43e38006f8cd6580ddabc4037905b2fad734712b582c']")
mal = Malware(labels=["rootkit"], name= "Alexios")

mem.add([ind2,ind3, mal])

# for visual purposes
print(mem.get(ind3.id))

{
    "type": "indicator",
    "id": "indicator--8f6df0be-6ada-4e2f-b711-886f355b3628",
    "created": "2017-09-24T19:32:32.359Z",
    "modified": "2017-09-24T19:32:32.359Z",
    "labels": [
        "malicious-activity"
    ],
    "description": "Crusades stage 2 implant variant",
    "pattern": "[file:hashes.'SHA-256' = '31a45e777e4d58b97f4c43e38006f8cd6580ddabc4037905b2fad734712b582c']",
    "valid_from": "2017-09-24T19:32:32.359374Z"
}


In [3]:
from stix2 import Filter

# add dictionary (of STIX object) to MemoryStore
# (this dict would assumably come from output of another source,
# i.e. a loaded json file, NOT manually created as done here for sample purposes)

malware = {
    "type": "malware",
    "id" : "malware--2daa14d6-cbf3-4308-bb8e-226d324a08e4",
    "labels": ["rootkit"],
    "name": "Urban2",
    "created": "2017-09-12T13:26:18.023Z",
    "modified": "2017-09-12T13:26:18.023Z"
}

mem.add(malware)

results = mem.query([Filter("labels","=", "rootkit")])
for r in results:
    # note that python STIX objects are pretty-printed
    # due to some python dunder method magic, but normal
    # python dictionaries are not by default. Thus the
    # python STIX objects and python STIX dictionaries
    # that match the above query can be easily identified visually
    print("-----------------------")
    print(r)

-----------------------
{
    "type": "malware",
    "id": "malware--b14b7ae7-5738-46db-bf8d-aec4ddd7c38c",
    "created": "2017-09-24T19:32:32.367Z",
    "modified": "2017-09-24T19:32:32.367Z",
    "name": "Alexios",
    "labels": [
        "rootkit"
    ]
}
-----------------------
{'name': 'Urban2', 'created': '2017-09-12T13:26:18.023Z', 'labels': ['rootkit'], 'modified': '2017-09-12T13:26:18.023Z', 'type': 'malware', 'id': 'malware--2daa14d6-cbf3-4308-bb8e-226d324a08e4'}


In [4]:
from stix2 import Filter

# add json formatted string to MemoryStore
# Again, would NOT manual create json-formatted string
# but taken as an output form from another source
report = '{"type": "report","id": "report--2add14d6-bbf3-4308-bb8e-226d314a08e4","labels": ["threat-report"], "name": "The Crusades: Looking into the relentless infiltration of Israels digital infrastructure.", "published": "2017-05-08T10:24:11.011Z", "object_refs":["malware--2daa14d6-cbf3-4308-bb8e-226d324a08e4"], "created": "2017-05-08T18:34:08.042Z", "modified": "2017-05-08T18:34:08.042Z"}'

mem.add(report)

print(mem.get("report--2add14d6-bbf3-4308-bb8e-226d314a08e4"))

{
    "type": "report",
    "id": "report--2add14d6-bbf3-4308-bb8e-226d314a08e4",
    "created": "2017-05-08T18:34:08.042Z",
    "modified": "2017-05-08T18:34:08.042Z",
    "name": "The Crusades: Looking into the relentless infiltration of Israels digital infrastructure.",
    "published": "2017-05-08T10:24:11.011Z",
    "object_refs": [
        "malware--2daa14d6-cbf3-4308-bb8e-226d324a08e4"
    ],
    "labels": [
        "threat-report"
    ]
}


### load_from_file() and save_to_file()

In [5]:
mem_2 = MemoryStore()

# save (dump) all STIX content in MemoryStore to json file
mem.save_to_file("path_to_target_file.json")

# load(add) STIX content from json file into MemoryStore
mem_2.load_from_file("path_to_target_file.json")

report = mem_2.get("report--2add14d6-bbf3-4308-bb8e-226d314a08e4")

# for visualpurposes
# Note: Since STIX content was added to MemoryStore as json,
# it is maintained as python dictionaries ( as opposed to STIX objects)
print(report)

{u'name': u'The Crusades: Looking into the relentless infiltration of Israels digital infrastructure.', u'created': u'2017-05-08T18:34:08.042Z', u'labels': [u'threat-report'], u'modified': u'2017-05-08T18:34:08.042Z', u'object_refs': [u'malware--2daa14d6-cbf3-4308-bb8e-226d324a08e4'], u'published': u'2017-05-08T10:24:11.011Z', u'type': u'report', u'id': u'report--2add14d6-bbf3-4308-bb8e-226d314a08e4'}
